Depedencies

In [1]:
!pip install transformers datasets trl torch accelerate bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import transformers
print(transformers.__version__)

4.51.3


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, pipeline
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
import os

device = ( "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

print(f"Device available : {device}")

Device available : cuda


In [8]:
!nvidia-smi

Fri May  9 09:13:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
model_name = "HuggingFaceTB/SmolLM2-360M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up chat format for the model and tokenizer
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Test the base model with a simple prompt
prompt = "Explain AGI ?"
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)
print(pipe(prompt, max_new_tokens=200))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

Device set to use cuda:0


[{'generated_text': 'Explain AGI ?\n\nA. AGI is a measure of the ability of a machine to perform a task.\n\nB. AGI is a measure of the ability of a machine to perform a task.\n\nC. AGI is a measure of the ability of a machine to perform a task.\n\nD. AGI is a measure of the ability of a machine to perform a task.\n\nAnswer: B\n\nQuestion 10. Which of the following is not a type of AGI?\n\nA. Machine learning\n\nB. Natural language processing\n\nC. Computer vision\n\nD. Speech recognition\n\nAnswer: B\n\nQuestion 11. Which of the following is not a type of AGI?\n\nA. Machine learning\n\nB. Natural language processing\n\nC. Computer vision\n\nD. Speech recognition\n\nAnswer: C\n\nQuestion 12. Which of the following is not a type of A'}]


In [10]:
finetuning_dataset = load_dataset("prithivMLmods/Deepthink-Reasoning")

# Tokenization function with batch handling
def tokenize_function(examples):
    # Apply strip to each string in the batch
    prompts = [p.strip() for p in examples["prompt"]]
    responses = [r.strip() for r in examples["response"]]

    # Apply chat template for each prompt-response pair
    texts = [
        tokenizer.apply_chat_template(
            [{"role": "user", "content": p}, {"role": "assistant", "content": r}],
            tokenize=False
        )
        for p, r in zip(prompts, responses)
    ]

    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

ds = finetuning_dataset.map(tokenize_function, batched=True)

README.md:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/401k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251 [00:00<?, ? examples/s]

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [12]:
use_bf16 = torch.cuda.is_bf16_supported()

# Prepare training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,  # Adjust max_steps as needed
    learning_rate=2e-4,
    fp16=not use_bf16,  # Use fp16 if bf16 is not supported
    bf16=use_bf16,  # Use bf16 if supported
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="none",  # Use 'wandb' or 'tensorboard' if needed or pass 'none'
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,  # Use tokenizer as processing_class
    train_dataset=ds["train"],  # Train dataset
    args=training_args,  # Pass the training arguments
)

In [13]:
trainer.train()

Step,Training Loss
1,1.840900
2,1.662400
3,1.509200
4,1.364100
5,1.350800
6,1.111400
7,1.244500
8,0.991700
9,1.139100
10,0.897600


TrainOutput(global_step=60, training_loss=0.7531551271677017, metrics={'train_runtime': 311.0631, 'train_samples_per_second': 1.543, 'train_steps_per_second': 0.193, 'total_flos': 459115683840000.0, 'train_loss': 0.7531551271677017})

In [ ]:
# Save model and tokenizer to a local directory
save_directory = "/content/SmoLLM/my_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Zip the directory
import shutil
shutil.make_archive(save_directory, 'zip', save_directory)

# Download the zip file
from google.colab import files
files.download(f"{save_directory}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the fine-tuned model
model_path = "/content/my_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create inference pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Test with a prompt
prompt = "Explain AGI in simple terms."
output = pipe(prompt, max_new_tokens=200, do_sample=True)
print(output[0]['generated_text'])


Device set to use cuda:0


Explain AGI in simple terms. Write explanation how a machine with AGI can think unlike humans and write about the challenges and benefits of machine thought. Provide an example of how AGI might have influenced social policies or technology.

## Summary
AI is a field of computer science that focuses on creating intelligent systems capable of performing tasks that would normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AGI refers to machines with the ability to reason, learn, and solve problems autonomously, often surpassing human performance in those domains. While still in its infancy, AGI holds great promise for transformative changes in various fields, including healthcare, education, and transportation.

AI will continue to revolutionize societal and professional landscapes through automation, personalized experiences, and improved efficiency. However, there are challenges associated with AGI creation, such